In [70]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from datetime import datetime
from imblearn.over_sampling import RandomOverSampler
from keras.models import Sequential
from keras.layers import Dense
import random
import h2o
from h2o.automl import H2OAutoML

In [71]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,20 mins 01 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_ekaushik_71hfe8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,190.7 Mb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [72]:
nsfas_df = pd.read_csv('C:/Users/Tau/Desktop/Repository/FraudGuardian/FraudGuardian/Financial analysis and fraud dection AI/data/nsfas_data.csv')  
university_df = pd.read_csv('C:/Users/Tau/Desktop/Repository/FraudGuardian/FraudGuardian/Financial analysis and fraud dection AI/data/university_data.csv')  
merged_df = pd.merge(nsfas_df, university_df, left_on='ID Number', right_on='Student ID', suffixes=('_nsfas', '_university'))


In [73]:
merged_df['Date of Birth_nsfas'] = pd.to_datetime(merged_df['Date of Birth_nsfas'])
merged_df['Age'] = ((datetime.now() - merged_df['Date of Birth_nsfas']).dt.total_seconds() / (365.25 * 24 * 3600)).astype(int)


In [74]:
X = merged_df[['Gender_nsfas', 'Age', 'Course_nsfas']]
y = merged_df['Financial Need']

In [75]:
label_encoder = LabelEncoder()
X['Gender_nsfas'] = label_encoder.fit_transform(X['Gender_nsfas'])
X['Course_nsfas'] = label_encoder.fit_transform(X['Course_nsfas'])

c:\Users\ekaushik\Desktop\Fraud Detection\venv3.8\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\ekaushik\Desktop\Fraud Detection\venv3.8\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [76]:
oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [78]:
train_frame = h2o.H2OFrame(pd.concat([X_train, pd.DataFrame({'target': y_train})], axis=1))
test_frame = h2o.H2OFrame(pd.concat([X_test, pd.DataFrame({'target': y_test})], axis=1))


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [79]:
predictors = X_train.columns.tolist()
response = 'target'

In [80]:
automl = H2OAutoML(max_runtime_secs=600)
automl.train(x=predictors, y=response, training_frame=train_frame)

AutoML progress: |█
15:46:23.888: AutoML: XGBoost is not available; skipping it.
15:46:24.194: GLM_1_AutoML_2_20240326_154623 [GLM def_1] failed: java.lang.ArrayIndexOutOfBoundsException: 3
15:46:24.197: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 7.0.
15:46:24.417: _min_rows param, The dataset size is too small to split for min_rows=10.0: must have at least 20.0 (weighted) rows, but have only 7.0.
15:46:24.418: _min_rows param, The dataset size is too small to split for min_rows=10.0: must have at least 20.0 (weighted) rows, but have only 7.0.
15:46:24.419: _min_rows param, The dataset size is too small to split for min_rows=10.0: must have at least 20.0 (weighted) rows, but have only 7.0.
15:46:27.398: StackedEnsemble_BestOfFamily_1_AutoML_2_20240326_154623 [StackedEnsemble best_of_family_3 (built with AUTO metalearner, using top model from each algorithm type)] failed: java.lang.RuntimeException:

In [ ]:
best_model = automl.leader

In [ ]:
y_pred = best_model.predict(test_frame).as_data_frame()['predict'].values

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%


c:\Users\ekaushik\Desktop\Fraud Detection\venv3.8\lib\site-packages\h2o\frame.py:1987: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  "    pandas_df = h2o_df.as_data_frame()\n", H2ODependencyWarning)


In [ ]:
unique_labels = y_test.unique()
print("Unique labels:", unique_labels)

if 'Low' in unique_labels:
    pos_label = 'Low'
elif 'Medium' in unique_labels:
    pos_label = 'Medium'
else:
    raise ValueError("No valid positive label found in y_test")

precision = precision_score(y_test, y_pred, pos_label=pos_label)
recall = recall_score(y_test, y_pred, pos_label=pos_label)


Unique labels: ['Low' 'Medium']


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='Medium', average='binary')
recall = recall_score(y_test, y_pred, pos_label='Medium', average='binary')
f1 = f1_score(y_test, y_pred, pos_label='Medium', average='binary')

print("Best Model by H2O AutoML:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Best Model by H2O AutoML:
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 0.6666666666666666
